In [26]:
import contextlib
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
import contextlib
import string
import nltk
import json

import spacy
import lemminflect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load pre-computed TF-IDFs and tokenized corpus
with open('data/fairytales_word_tf-idfs.json', 'r') as f:
    tf_idfs = json.load(f)
with open('data/fairytales_tokenized.json', 'r') as f:
    tokenized_corpus = json.load(f)

print(f"Loaded {len(tf_idfs)} words with TF-IDFs")
print(f"Loaded {len(tokenized_corpus)} tokenized sentences")

Loaded 18157 words with TF-IDFs
Loaded 142049 tokenized sentences


In [27]:
# Skip - data already loaded from JSON files above

In [28]:
# Skip - functions not needed

In [ ]:
# Skip - corpus already processed and saved. Data loaded in cell 1.

In [ ]:
with open('data/n_neighbors.json', 'r') as f:
    n_neighbors = json.load(f)
with open('data/neighbor_frequencies.json', 'r') as f:
    neighbor_frequencies = json.load(f)

In [ ]:
import copy
import math
tf_idfs = copy.deepcopy(neighbor_frequencies) # make copy of the relative frequencies json to convert frequencies to tf-idfs
for word in neighbor_frequencies.keys(): # ex: "queen"
    for neighbor in neighbor_frequencies[word].keys(): # The words next to "queen", i.e. "lady", "fairy", "king", etc.
        tf_idfs[word][neighbor] = math.log(neighbor_frequencies[word][neighbor] / n_neighbors[word] * math.log(words_in_corpus / n_neighbors[neighbor]))

for word in tf_idfs.keys():
    tf_idfs[word] = dict(sorted(tf_idfs[word].items(), key=lambda item: item[1], reverse=True)) # sorts TF-IDFs in decreasing order

In [ ]:
import pandas as pd
import numpy as np
dist = np.concatenate([list(x.values()) for x in tf_idfs.values()])
df = pd.Series(dist)
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
(df > (Q3 + 1.5 * IQR)).sum()

ValueError: need at least one array to concatenate

In [ ]:
minimum = df.min()
df.describe()

count    2.871242e+06
mean    -4.928283e+00
std      1.728402e+00
min     -9.715406e+00
25%     -6.144078e+00
50%     -5.009849e+00
75%     -3.843962e+00
max      2.597717e+00
dtype: float64

In [ ]:
for word in tf_idfs.keys():
    for neighbor in tf_idfs[word].keys():
        tf_idfs[word][neighbor] -= minimum

In [ ]:
with open('data/fairytales_word_tf-idfs.json', 'w') as f:
    json.dump(tf_idfs, f, indent=4)
with open('data/fairytales_tokenized.json', 'w') as f:
    json.dump(tokenized_corpus, f, indent=4)

In [ ]:
import matplotlib.pyplot as plt
def plot_tf_idf_distribution(tf_idfs, title, bounds=(0,1)):
    all_values = []
    for word in tf_idfs.keys():
        all_values.extend(tf_idfs[word].values())

    plt.figure(figsize=(10, 6))
    plt.hist(all_values, bins=50, range=bounds, edgecolor='black')
    plt.title(title)
    plt.xlabel('TF-IDF')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()

In [1]:
plot_tf_idf_distribution(tf_idfs, 'TF-IDFs', bounds=(0, 13))

NameError: name 'plot_tf_idf_distribution' is not defined